In [1]:
# -*- coding:utf-8 -*-

In [2]:
import os
import sys
#import cv2
import math
import numpy as np
import pandas as pd
from skimage import filters,io
#import matplotlib.pyplot as plt

In [3]:
filename = "./datasets/Char_Index.txt"
INPUT_DIR = "./datasets/Char_Image"
OUTPUT_DIR = "./outputs/Char_Image_Binary"

##  PreProcessing

In [4]:
input1 = []
input2 = []
input3 = []
#Efield = []
with open(filename, 'r') as  file_to_read:
    while True:
        # 整行读取数据
        lines = file_to_read.readline()
        if not lines:
            break
            pass
        # 将整行数据分割处理，这里分隔符为空格
        tmp1, tmp2, tmp3 = [i for i in lines.split()]
        input1.append(tmp1)
        input2.append(tmp2)
        input3.append(tmp3)
    input1 = np.array(input1[1:])
    input2 = np.array(input2[1:])
    input3 = np.array(input3[1:])
indexFileName = input3
input_y = input2

for k in range(0, len(indexFileName)):
    path = os.path.join(INPUT_DIR, indexFileName[k])
    image = io.imread(path, as_gray=True)
    #image = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2GRAY)
    thresh = filters.threshold_otsu(image)
    # 乘以1.0是因为将True,False转换为浮点数
    dst = (image > thresh) * 1
    a, b = dst.shape
    # # 绘图显示
    # plt.figure("thresh", figsize=(8,8))
    #
    # plt.subplot(121)
    # plt.title("Original")
    # plt.imshow(image, plt.cm.gray)
    #
    # plt.subplot(122)
    # plt.title("binary")
    # # 0表示黑,1表示白
    # plt.imshow(dst, plt.cm.gray)
    # plt.show()

    # 四个角的二值化数据大于2,白底
    if (dst[0,0] + dst[0, b-1] + dst[a-1, 0] + dst[a-1,b-1] >= 2):
        for i in range(0,a):
            for j in range(0,b):
                # 将每一个像素的值反向,转换成黑底白字
                dst[i,j] = 1 - dst[i,j]
    if not os.path.exist(OUTPUT_DIR):
        os.mkdir(OUTPUT_DIR)
    path_out = os.path.join(OUTPUT_DIR, str(k+1)+".jpg")
    dst = dst.astype(np.uint8) * 255
    io.imsave(path_out, dst)
    #io.imshow(dst)

## PreProcessingToCorrect 

In [5]:
input1_ = []
input2_ = []
with open("./datasets/Char_Index_Err.txt") as file_idx_err:
    while True:
        # 整行读取数据
        lines = file_idx_err.readline()
        if not lines:
            break
        # 将整行数据分割处理，这里分隔符为空格
        tmp1, tmp2 = [i for i in lines.split()]
        input1_.append(tmp1)
        input2_.append(tmp2)

    input1_ = np.array(input1_[1:])
    input2_ = np.array(input2_[1:])
    IndexFileName_Err = input2_

for k in range(0, len(IndexFileName_Err)):
    path_e = os.path.join(OUTPUT_DIR, IndexFileName_Err[k])
    img_e = io.imread(path_e)
    a, b = img_e.shape
    for i in range(0,a):
        for j in range(0,b):
            dst[i,j] = 255 - img_e[i,j]       
    io.imsave(path_e ,dst)
    #io.imshow(img_e)  

## Feature Extration

### Feature1

找出每一行\每一列的白色点的个数

In [6]:
with open("./outputs/feature1.txt",'w+') as fd:
    for k in range(0, 1000):
        path = os.path.join(OUTPUT_DIR, str(k+1)+'.jpg')
        image = io.imread(path)
        thresh = filters.threshold_otsu(image)
        dst = (image > thresh) * 1
        a, b = dst.shape
        C = np.zeros(a+b, dtype=np.int)

        # 每一行
        for i in range(0, a):
            for j in range(0, b):
                if(dst[i,j] == 1):
                    C[i] += 1
        # 每一列
        for j in range(0, b):
            for i in range(0, a):
                if(dst[i,j] == 1):
                    C[a+j] += 1

        fd.write(str(k+1) + '\t')
        for i in range(0, a+b-1):
            fd.write(str(C[i]) + ',')
        if k < 999:
            fd.write(str(C[a+b-1]) + '\n')
        else:
            fd.write(str(C[a+b-1]))


### Feature2

计算区域密度, 8*8

In [7]:
square8 = 8
with open("./outputs/feature2.txt", 'w+') as fd:
    for k in range(0, 1000):
        path = os.path.join(OUTPUT_DIR, str(k+1)+'.jpg')
        image = io.imread(path)
        thresh = filters.threshold_otsu(image)
        dst = (image > thresh) * 1
        a, b = dst.shape
        bh = math.ceil(a / square8)
        bw = math.ceil(b / square8)
        size_ = bh * bw
        # 定义特征向量
        C = np.zeros(size_, dtype=np.int)
        
        for i in range(0, a):
            x = math.floor(i / 8.0)
            for j in range(0, b):
                if dst[i, j] == 1:
                    y = math.floor(j / 8.0)
                    C[bw*x + y] += 1
                    
        fd.write(str(k+1) + '\t')
        for i in range(0, size_ - 1):
            fd.write(str(C[i]) + ',')
        if k < 999:
            fd.write(str(C[size_ - 1]) + '\n')
        else:
            fd.write(str(C[size_ - 1]))

### Feature3

计算区域密度 4*4

In [8]:
square4 = 4
with open("./outputs/feature3.txt", 'w+') as fd:
    for k in range(0, 1000):
        path = os.path.join(OUTPUT_DIR, str(k+1)+'.jpg')
        image = io.imread(path)
        thresh = filters.threshold_otsu(image)
        dst = (image > thresh) * 1
        a, b = dst.shape
        bh = math.ceil(a / square4)
        bw = math.ceil(b / square4)
        size_ = bh * bw
        # 定义特征向量
        C = np.zeros(size_, dtype=np.int)
        
        for i in range(0, a):
            x = math.floor(i / 8.0)
            for j in range(0, b):
                if dst[i, j] == 1:
                    y = math.floor(j / 8.0)
                    C[bw*x + y] += 1
                    
        fd.write(str(k+1) + '\t')
        for i in range(0, size_ - 1):
            fd.write(str(C[i]) + ',')
        if k < 999:
            fd.write(str(C[size_ - 1]) + '\n')
        else:
            fd.write(str(C[size_ - 1]))

### Feature4
计算区域密度 6*6

In [9]:
square6 = 6
with open("./outputs/feature4.txt", 'w+') as fd:
    for k in range(0, 1000):
        path = os.path.join(OUTPUT_DIR, str(k+1)+'.jpg')
        image = io.imread(path)
        thresh = filters.threshold_otsu(image)
        dst = (image > thresh) * 1
        a, b = dst.shape
        bh = math.ceil(a / square6)
        bw = math.ceil(b / square6)
        size_ = bh * bw
        # 定义特征向量
        C = np.zeros(size_, dtype=np.int)
        
        for i in range(0, a):
            x = math.floor(i / 8.0)
            for j in range(0, b):
                if dst[i, j] == 1:
                    y = math.floor(j / 8.0)
                    C[bw*x + y] += 1
                    
        fd.write(str(k+1) + '\t')
        for i in range(0, size_ - 1):
            fd.write(str(C[i]) + ',')
        if k < 999:
            fd.write(str(C[size_ - 1]) + '\n')
        else:
            fd.write(str(C[size_ - 1]))

### Feature5

计算每一行,每一列的线段数目

In [10]:
with open("./outputs/feature5.txt", 'w+') as fd:
    for k in range(0, 1000):
        path = os.path.join(OUTPUT_DIR, str(k+1)+'.jpg')
        image = io.imread(path)
        thresh = filters.threshold_otsu(image)
        dst = (image > thresh) * 1
        a, b = dst.shape
        size_ = a + b
        # 定义特征向量
        C = np.zeros(size_, dtype=np.int)
        
        for i in range(a):
            for j in range(b-1):
                if dst[i, j] != dst[i, j+1] and dst[i, j+1] == 1:
                    C[i] += 1
            
        for j in range(b):
            for i in range(a-1):
                if dst[i, j] != dst[i+1, j] and dst[i+1, j] == 1:
                    C[a+j] += 1
                    
        fd.write(str(k+1) + '\t')
        for i in range(0, size_ - 1):
            fd.write(str(C[i]) + ',')
        if k < 999:
            fd.write(str(C[size_ - 1]) + '\n')
        else:
            fd.write(str(C[size_ - 1]))

### Feature6

计算字符与上,下,左,右四个边界之间的距离

## 特征融合

In [11]:
feature1 = []
with open("./outputs/feature1.txt", 'r') as fp:
    for line in fp:
        linearr = line.strip().split('\t')
        feature1.append(linearr[1].split(','))
    feature1 = np.array(feature1)

feature2 = []
with open("./outputs/feature2.txt", 'r') as fp:
    for line in fp:
        linearr = line.strip().split('\t')
        feature2.append(linearr[1].split(','))
    feature2 = np.array(feature2)  

feature3 = []
with open("./outputs/feature3.txt", 'r') as fp:
    for line in fp:
        linearr = line.strip().split('\t')
        feature3.append(linearr[1].split(','))
    feature3 = np.array(feature3)

feature4 = []
with open("./outputs/feature4.txt", 'r') as fp:
    for line in fp:
        linearr = line.strip().split('\t')
        feature4.append(linearr[1].split(','))
    feature4 = np.array(feature4)

feature5 = []
with open("./outputs/feature5.txt", 'r') as fp:
    for line in fp:
        linearr = line.strip().split('\t')
        feature5.append(linearr[1].split(','))
    feature5 = np.array(feature5)

In [12]:
input_x = np.append(
               np.append(
                   np.append(
                       np.append(feature1, 
                           feature2, axis=1),
                           feature3, axis=1),  
                           feature4, axis=1),
                           feature5, axis=1)

In [13]:
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split

## 模型训练

In [14]:
# 为了看模型在没有见过数据集上的表现，随机拿出数据集中30%的部分做测试
X_train, X_test, y_train, y_test = train_test_split(input_x, input_y, test_size=0.3, random_state=0) 

In [15]:
svc = svm.SVC(kernel='poly', degree=3, gamma=1, coef0=0)
# 训练模型
clf = svc.fit(X_train, y_train)
# 计算测试集精度
score = clf.score(X_test, y_test)
print('精度为%s' % score)

精度为0.9966666666666667
